In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers torch

In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install tensorboard

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from torch.utils.data import Dataset
import os
os.environ["WANDB_MODE"] = "disable"

In [ ]:
train_data = pd.read_excel('/content/drive/MyDrive/DPB4_Claudia Camacho/code/datasets_finales/processed_training_data_v2.xlsx')
X = train_data['situacion']  # Texto
y = train_data.iloc[:, 1:]   # Etiquetas (columnas binarias)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
distortion_columns = train_data.columns[1:]

frecuencia_distorsiones = train_data[distortion_columns].sum().reset_index()
frecuencia_distorsiones.columns = ['Distorsión', 'Frecuencia']

frecuencia_distorsiones

,Distorsión,Frecuencia
0,pensamiento de todo o nada,1563
1,sobregeneralización,1625
2,etiquetado,1700
3,error del adivino,1500
4,lectura de la mente,1577
5,razonamiento emocional,1584
6,debería,1544
7,personalización,1564
8,comparación y desesperanza,1509
9,culpabilización,1511


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

def tokenize_data(texts, labels):
    encoding = tokenizer(
        texts.tolist(),
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors='pt'
    )
    encoding['labels'] = torch.tensor(labels.values, dtype=torch.float)
    return encoding

train_encodings = tokenize_data(X_train, y_train)
val_encodings = tokenize_data(X_val, y_val)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

In [ ]:
class CognitiveDistortionDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

train_dataset = CognitiveDistortionDataset(train_encodings)
val_dataset = CognitiveDistortionDataset(val_encodings)


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "dccuchile/bert-base-spanish-wwm-cased",
    num_labels=14  # 14 etiquetas para las distorsiones
)

training_args = TrainingArguments(
    output_dir='./results',           # Carpeta de salida
    evaluation_strategy="epoch",      # Evaluar en cada epoch
    learning_rate=1e-5,               # Tasa de aprendizaje
    per_device_train_batch_size=8,    # Tamaño del batch para entrenamiento
    per_device_eval_batch_size=8,     # Tamaño del batch para evaluación
    num_train_epochs=6,               # Número de épocas
    weight_decay=0.1,                # Decaimiento del peso
    report_to="none"                  # Desactivar reporte a wandb y otros
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = torch.sigmoid(torch.tensor(pred.predictions)) > 0.5  # Sigmoid y umbral de 0.5
    f1_micro = f1_score(labels, preds, average='micro')
    f1_macro = f1_score(labels, preds, average='macro')
    accuracy = accuracy_score(labels, preds)
    return {
        'accuracy': accuracy,
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

<ipython-input-9-60aaa35ddacc>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro
1,0.141400,0.134386,0.469605,0.608428,0.561945
2,0.115700,0.124003,0.545311,0.648838,0.626703
3,0.095400,0.122658,0.584859,0.660892,0.648141
4,0.082900,0.128760,0.591412,0.652990,0.644272
5,0.068700,0.135327,0.599548,0.649976,0.643422
6,0.062300,0.137110,0.603164,0.648904,0.644892


<ipython-input-9-60aaa35ddacc>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-9-60aaa35ddacc>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-9-60aaa35ddacc>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-9-60aaa35ddacc>:6: UserWarning: To copy construct from a tens

TrainOutput(global_step=13278, training_loss=0.10213435322858785, metrics={'train_runtime': 3512.7318, 'train_samples_per_second': 30.228, 'train_steps_per_second': 3.78, 'total_flos': 6985167042935808.0, 'train_loss': 0.10213435322858785, 'epoch': 6.0})

In [ ]:
model.save_pretrained("/content/drive/MyDrive/DPB4_Claudia Camacho/code/saved_model")
tokenizer.save_pretrained("/content/drive/MyDrive/DPB4_Claudia Camacho/code/saved_token")
#tokenizer = AutoTokenizer.from_pretrained("/ruta/donde/guardar/el/tokenizador")

### Validación